In [2]:
%pylab inline
import sys
sys.path.append("..")
from loaders import imagenet_load
from importlib import reload
print(imagenet_load)
from conv import multigpu
from loaders.imagenet_load import orient
import conv
from conv import filter_gen, multigpu
from opt import ls
import numpy as np
from utils import misc
from utils import linalg
from utils import exputil
from numba import jit
import math
import concurrent.futures as fs
from pylab import imshow
import copy
from skimage.filters import gabor_kernel
from skimage.transform import rescale
from sklearn import metrics

Populating the interactive namespace from numpy and matplotlib
<module 'loaders.imagenet_load' from '../loaders/imagenet_load.py'>


In [3]:
NUM_CLASSES = 100

In [3]:

def class_to_tarball_name_train(classname):
    classname = classname.strip()
    return "/home/ubuntu/imagenet-train/{0}-scaled.tar".format(classname)

def class_to_tarball_name_val(classname):
    classname = classname.strip()
    return "/home/ubuntu/imagenet-validation/val-{0}-scaled.tar".format(classname)



In [31]:
def normalize_images(images, min_divisor=1e-8):
    orig_shape = images.shape
    images = images.reshape(images.shape[0], -1)
    print("PRE PROCESSING")
    n_images = images.shape[0]
    # Zero mean every feature
    images = images - np.mean(images, axis=1)[:,np.newaxis]
    # Normalize
    image_norms = np.linalg.norm(images, axis=1)/55.0
    # Get rid of really small norms
    image_norms[np.where(image_norms < min_divisor)] = 1
    # Make features unit norm
    images_normalized = images/image_norms[:,np.newaxis]
    return images_normalized.reshape(orig_shape)

In [4]:
%time loader_train = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../loaders/classes", tarball_func=class_to_tarball_name_train, mmap_loc="/dev/shm/imagenet_train", n_procs=16)
%time loader_val = imagenet_load.ImagenetLoader(NUM_CLASSES, classes_path="../loaders/classes", tarball_func=class_to_tarball_name_val, mmap_loc="/dev/shm/imagenet_val", n_procs=16)

%time X_train = loader_train.load_all()
%time X_test = loader_val.load_all()

There exists a matrix at /dev/shm/imagenet_train


CPU times: user 36 ms, sys: 2.24 s, total: 2.28 s
Wall time: 13.7 s


There exists a matrix at /dev/shm/imagenet_val


CPU times: user 28 ms, sys: 216 ms, total: 244 ms
Wall time: 433 ms
Loaded 0 classes into mem
CPU times: user 80 ms, sys: 76 ms, total: 156 ms
Wall time: 18.8 s
Loaded 0 classes into mem
CPU times: user 40 ms, sys: 88 ms, total: 128 ms
Wall time: 902 ms


In [8]:
%time X_train = X_train.astype('float32')
%time X_test = X_test.astype('float32')

CPU times: user 29 s, sys: 24.5 s, total: 53.5 s
Wall time: 53.5 s
CPU times: user 1.12 s, sys: 932 ms, total: 2.06 s
Wall time: 2.06 s


In [11]:
X_train.shape

(129395, 3, 256, 256)

In [12]:
#%time X_train = normalize_images(X_train)
#%time X_test = normalize_images(X_test)

%time mean_image = np.mean(X_train, axis=0)
%time X_train -= mean_image
%time X_test -= mean_image

CPU times: user 16 s, sys: 0 ns, total: 16 s
Wall time: 16 s
CPU times: user 16.1 s, sys: 0 ns, total: 16.1 s
Wall time: 16.1 s
CPU times: user 624 ms, sys: 0 ns, total: 624 ms
Wall time: 622 ms


In [32]:
X_train.filename = "/dev/shm/X_train"
X_test.filename = "/dev/shm/X_test"

In [13]:
y_train = loader_train.Y
y_test = loader_val.Y

In [14]:
num_pixels = X_train.shape[1]*X_train.shape[2]*X_train.shape[3]

In [ ]:
num_pixels

In [18]:
%time X_train_flat = X_train.reshape(X_train.shape[0], -1) / np.sqrt(num_pixels)
%time X_test_flat = X_test.reshape(X_test.shape[0], -1)  / np.sqrt(num_pixels)

CPU times: user 4.07 s, sys: 4.5 s, total: 8.58 s
Wall time: 8.57 s
CPU times: user 156 ms, sys: 176 ms, total: 332 ms
Wall time: 330 ms


In [19]:
X_train_flat

array([[ -2.03486261e-04,  -2.20109386e-04,  -1.79659784e-04, ...,
          2.96890380e-04,   3.02852941e-04,   3.03848768e-04],
       [ -5.44885060e-05,  -7.43944000e-05,  -8.84689705e-05, ...,
         -4.26344990e-04,  -3.78707015e-04,  -3.23238227e-04],
       [ -1.14394453e-04,  -1.16142693e-04,  -1.16599023e-04, ...,
          7.24585938e-04,   7.26829778e-04,   7.27825605e-04],
       ..., 
       [  1.54596231e-04,   2.10270672e-04,   8.34844417e-05, ...,
         -3.36185703e-04,  -2.88003717e-04,  -3.21461499e-04],
       [  3.04049863e-04,   2.72234284e-04,   2.86811623e-04, ...,
         -6.31025631e-05,  -9.59372845e-05,  -1.45053689e-04],
       [ -2.93694146e-04,  -5.33300616e-05,   2.58411080e-04, ...,
         -7.02395444e-05,  -1.18966720e-04,  -4.78857462e-05]])

In [20]:
K_train_flat = X_train_flat.dot(X_train_flat.T)
K_test_flat = X_test_flat.dot(X_train_flat.T)

In [21]:
np.mean(X_train_flat, axis=1)

array([ -6.21157495e-21,  -6.77626358e-21,  -9.03501810e-21, ...,
         2.37169225e-20,  -5.19513541e-20,  -9.03501810e-21])

In [22]:
results_flat = {}

In [23]:
for reg in [1e-4, 1e-2, 1e-1, 1e1, 10, 1000, 10000]:
    acc = opt.trainAndEvaluateDualModel(KTrain=K_train_flat, KTest=K_test_flat, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
    print(reg, acc)
    results_flat[(0,reg)] = acc

Learning Dual Model
reg is 0.0001
0.0001 (0.99761538461538457, 0.23000000000000001)
Learning Dual Model
reg is 0.01
0.01 (0.8223076923076923, 0.34599999999999997)
Learning Dual Model
reg is 0.1
0.1 (0.52746153846153843, 0.376)
Learning Dual Model
reg is 10.0
10.0 (0.34069230769230768, 0.32200000000000001)
Learning Dual Model
reg is 10
10 (0.34069230769230768, 0.32200000000000001)
Learning Dual Model
reg is 1000
1000 (0.29215384615384615, 0.27000000000000002)
Learning Dual Model
reg is 10000
10000 (0.29069230769230769, 0.27200000000000002)


In [28]:
for gamma in [0.1, 1, 1e2, 1e3]:
    K_train_rbf_pixels = linalg.computeRBFGramMatrix(X_train_flat, X_train_flat, gamma=gamma)
    K_test_rbf_pixels = linalg.computeRBFGramMatrix(X_test_flat, X_train_flat, gamma=gamma)
    for reg in [1e-5, 1e-4, 1e-1, 1, 10, 100]:
        acc = opt.trainAndEvaluateDualModel(KTrain=K_train_rbf_pixels, KTest=K_test_rbf_pixels, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
        print(gamma, reg, acc)
        results_flat[(gamma,reg)] = acc

Learning Dual Model
reg is 1e-05
0.1 1e-05 (0.99792307692307691, 0.28000000000000003)
Learning Dual Model
reg is 0.0001
0.1 0.0001 (0.98869230769230765, 0.33600000000000002)
Learning Dual Model
reg is 0.1
0.1 0.1 (0.42253846153846153, 0.39600000000000002)
Learning Dual Model
reg is 1
0.1 1 (0.35861538461538461, 0.35199999999999998)
Learning Dual Model
reg is 10
0.1 10 (0.30599999999999999, 0.28399999999999997)
Learning Dual Model
reg is 100
0.1 100 (0.29261538461538461, 0.27200000000000002)
Learning Dual Model
reg is 1e-05
1 1e-05 (0.99807692307692308, 0.29199999999999998)
Learning Dual Model
reg is 0.0001
1 0.0001 (0.99792307692307691, 0.32600000000000001)
Learning Dual Model
reg is 0.1
1 0.1 (0.61969230769230765, 0.40400000000000003)
Learning Dual Model
reg is 1
1 1 (0.42884615384615382, 0.40400000000000003)
Learning Dual Model
reg is 10
1 10 (0.35976923076923079, 0.35399999999999998)
Learning Dual Model
reg is 100
1 100 (0.30615384615384617, 0.28399999999999997)
Learning Dual Model


In [15]:
def normalize_patches(patches, min_divisor=1e-8, zca_bias=0.1):

    orig_shape = patches.shape
    patches = patches.reshape(patches.shape[0], -1)
    print("PRE PROCESSING")
    n_patches = patches.shape[0]

    # Zero mean every feature
    patches = patches - np.mean(patches, axis=1)[:,np.newaxis]

    # Normalize
    patch_norms = np.linalg.norm(patches, axis=1)/55.0

    # Get rid of really small norms
    patch_norms[np.where(patch_norms < min_divisor)] = 1

    # Make features unit norm
    patches = patches/patch_norms[:,np.newaxis]


    whitening_means = np.mean(patches, axis=0)
    
    data_means = np.mean(patches, axis=1)


    zeroCenterPatches = (patches - whitening_means[np.newaxis, :])

    patchesCovMat = 1.0/n_patches * zeroCenterPatches.T.dot(zeroCenterPatches)

    (E,V) = np.linalg.eig(patchesCovMat)

    E += zca_bias
    sqrt_zca_eigs = np.sqrt(E)
    
    inv_sqrt_zca_eigs = np.diag(np.power(sqrt_zca_eigs, -1))
    
    global_ZCA = V.dot(inv_sqrt_zca_eigs).dot(V.T)
    
    patches_normalized = (patches).dot(global_ZCA)

    return patches_normalized.reshape(orig_shape)




@jit(nopython=True, nogil=True, cache=True)
def __grab_patches(images, patch_size=6, tot_patches=1e6, seed=0, scale_pixels=1.0/255.0):
    np.random.seed(seed)
    tot_patches = int(tot_patches)
    im_idxs = np.random.choice(images.shape[0], tot_patches)
    idxs_x = np.random.choice(images.shape[3] - patch_size - 1, tot_patches)
    idxs_y = np.random.choice(images.shape[2] - patch_size - 1, tot_patches)
    idxs_x += int(math.ceil(patch_size/2))
    idxs_y += int(math.ceil(patch_size/2))
    patches = np.zeros((tot_patches, images.shape[1], patch_size, patch_size), dtype=np.float32)
    for i, (im_idx, idx_x, idx_y) in enumerate(zip(im_idxs, idxs_x, idxs_y)):
        out_patch = patches[i, :, :, :]
        grab_patch_from_idx(images[im_idx], idx_x, idx_y, patch_size, out_patch, scale_pixels)
    return patches
    
@jit(nopython=True, nogil=True)
def grab_patch_from_idx(im, idx_x, idx_y, patch_size, outpatch, scale_pixels=1.0/255.0):
    sidx_x = int(idx_x - patch_size/2)
    eidx_x = int(idx_x + patch_size/2)
    sidx_y = int(idx_y - patch_size/2)
    eidx_y = int(idx_y + patch_size/2)
    outpatch[:,:,:] = im[:, sidx_x:eidx_x, sidx_y:eidx_y,].astype(np.float32)*scale_pixels
    return outpatch

def grab_patches(images, patch_size=6, tot_patches=1e6, seed=0, max_threads=50, scale_pixels=1.0/255.0):
    idxs = misc.chunk_idxs(images.shape[0], max_threads)
    tot_patches = int(tot_patches)
    patches_per_thread = int(tot_patches/max_threads)
    with fs.ThreadPoolExecutor(max_workers=max_threads) as executor:
        futures = []
        for i,(sidx, eidx) in enumerate(idxs):
            futures.append(executor.submit(__grab_patches, images[sidx:eidx, :], 
                                           patch_size=patch_size,
                                           tot_patches=patches_per_thread,
                                           seed=seed+i))
            
        results = np.vstack(list(map(lambda x: x.result(), futures)))
    return results 

In [16]:
patch_size = 12
%time patches = grab_patches(X_train, patch_size=patch_size)

CPU times: user 25 s, sys: 1.25 s, total: 26.3 s
Wall time: 3.02 s


In [17]:
patches_normalized = normalize_patches(patches)

PRE PROCESSING


In [18]:
reload(filter_gen)
patches_mmap_data = np.memmap("/dev/shm/imagenet_patches", shape=patches.shape, dtype=patches.dtype, mode="w+")
np.copyto(patches_mmap_data, patches)
patches_mmap = multigpu.MmapArray(patches_mmap_data, mode="r+")

patches_normalized_mmap_data = np.memmap("/dev/shm/imagenet_patches_normalized", shape=patches.shape, dtype=patches.dtype, mode="w+")
np.copyto(patches_normalized_mmap_data, patches_normalized)
patches_normalized_mmap = multigpu.MmapArray(patches_normalized_mmap_data, mode="r+")

fg_patches = filter_gen.make_empirical_filter_gen(patches_mmap, seed=0)
fg_patches_normalized = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, seed=0)

In [20]:
gpu_handler = multigpu.MultiGpuHandler(16)
gpu_handler.kill_all_gpu_processes()
gpu_handler.start_and_wait_for_gpu_init()

In [90]:
conv_args_template = \
{ 
  "num_feature_batches":5,
  "data_batch_size":8,
  "feature_batch_size":1024,
  "pool_size":70,
  "pool_type":"avg",
  "pool_stride":45,
  "patch_size":patch_size,
  "pad":0,
  "bias":0,
  "conv_stride":2,
  "scale_pixels": 1.0/255.0
}
%time X_train_normalized = np.memmap("/dev/shm/X_train_normalized", dtype='float32', shape=X_train.shape, mode="w+")
%time X_test_normalized = np.memmap("/dev/shm/X_test_normalized", dtype='float32', shape=X_test.shape, mode="w+")

%time np.copyto(X_train_normalized, X_train.astype('float32'))
%time np.copyto(X_test_normalized, X_test.astype('float32'))

X_train_mmap = multigpu.MmapArray(X_train_normalized, mode="r+")
X_test_mmap = multigpu.MmapArray(X_test_normalized, mode="r+")

mmap_out_train_shape = conv.conv_compute_output_shape(data=X_train_mmap, **conv_args_template)
X_train_lift = np.memmap("/dev/shm/X_train_lift", mode="w+", dtype="float32", shape=mmap_out_train_shape)

mmap_out_test_shape = conv.conv_compute_output_shape(data=X_test_mmap, **conv_args_template)
X_test_lift = np.memmap("/dev/shm/X_test_lift", mode="w+", dtype="float32", shape=mmap_out_test_shape)

CPU times: user 0 ns, sys: 10.3 s, total: 10.3 s
Wall time: 10.3 s
CPU times: user 0 ns, sys: 416 ms, total: 416 ms
Wall time: 418 ms
CPU times: user 39.8 s, sys: 1min 20s, total: 2min
Wall time: 2min
CPU times: user 1.48 s, sys: 2.69 s, total: 4.17 s
Wall time: 4.19 s


In [91]:
X_train_mmap.loc

'/dev/shm/X_train_normalized'

In [92]:
fg_patches = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, MIN_VAR_TOL=0.1, seed=0)

In [93]:
np.mean(np.var(fg_patches(1024), axis=(1,2,3)))

1.5557241

In [94]:
X_train_lift_mmap = multigpu.MmapArray(X_train_lift, mode="r+")
X_test_lift_mmap = multigpu.MmapArray(X_test_lift, mode="r+")


In [95]:
train_chunk_idxs = misc.chunk_idxs(X_train.shape[0], 16)
test_chunk_idxs = misc.chunk_idxs(X_test.shape[0], 16)

In [96]:
gpu_results = []
for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus, train_chunk_idxs)):
        fg_patches = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, MIN_VAR_TOL=0.1, seed=0)
        conv_args = conv_args_template.copy()
        conv_args["filter_gen"] = fg_patches
        X_train_mmap_gpu = copy.copy(X_train_mmap)
        X_train_mmap_gpu.idxs = (sidx, eidx)
        X_train_out_gpu = copy.copy(X_train_lift_mmap)
        X_train_out_gpu.idxs = (sidx, eidx)
        conv_args['data'] = X_train_mmap_gpu
        conv_args['output'] = X_train_out_gpu
        gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
        gpu_results.append(gpu_result)

for gpu_result in gpu_results:
    %time gpu_result.result()

CPU times: user 28 ms, sys: 20 ms, total: 48 ms
Wall time: 14min 24s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 629 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 30.2 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 728 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 540 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 552 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 470 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 516 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 522 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 502 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 491 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 432 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 360 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 420 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 489 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall

In [97]:
gpu_results = []
for gpu, (sidx, eidx) in list(zip(gpu_handler.gpus, test_chunk_idxs)):
        fg_patches = filter_gen.make_empirical_filter_gen(patches_normalized_mmap, MIN_VAR_TOL=0.1, seed=0)
        conv_args = conv_args_template.copy()
        conv_args["filter_gen"] = fg_patches
        X_test_mmap_gpu = copy.copy(X_test_mmap)
        X_test_mmap_gpu.idxs = (sidx, eidx)
        X_test_out_gpu = copy.copy(X_test_lift_mmap)
        X_test_out_gpu.idxs = (sidx, eidx)
        conv_args['data'] = X_test_mmap_gpu
        conv_args['output'] = X_test_out_gpu
        gpu_result = gpu.submit_async(conv.conv_mmap, **conv_args)
        gpu_results.append(gpu_result)

for gpu_result in gpu_results:
    %time gpu_result.result()

CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 32.7 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 610 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 1.41 s
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 214 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 370 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 329 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 326 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 326 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 327 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 334 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 324 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 372 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 308 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 305 µs
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 306 µs
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time:

In [106]:
X_train_lift = X_train_lift_mmap.load()
X_test_lift = X_test_lift_mmap.load()

In [99]:
np.mean(np.sum(X_train_lift == 0, axis=(1,2,3)))/(len(X_train_lift[0].ravel()))

0.003517104743067094

In [100]:
X_train_lift.shape

(129395, 10240, 3, 3)

In [101]:
num_feats = conv_args_template['feature_batch_size']*conv_args_template['num_feature_batches']

In [102]:
%time X_train_lift_flat = X_train_lift.reshape(X_train_lift.shape[0], -1).astype('float64') / np.sqrt(num_feats)

CPU times: user 34.5 s, sys: 1min 6s, total: 1min 40s
Wall time: 1min 40s


In [103]:
%time X_test_lift_flat = X_test_lift.reshape(X_test_lift.shape[0], -1).astype('float64') / np.sqrt(num_feats)

CPU times: user 1.2 s, sys: 1.79 s, total: 2.98 s
Wall time: 2.99 s


In [1186]:
%time K_train_conv = X_train_lift_flat.dot(X_train_lift_flat.T) 
%time K_test_conv = X_test_lift_flat.dot(X_train_lift_flat.T)

CPU times: user 1min 53s, sys: 55.4 s, total: 2min 49s
Wall time: 6.01 s
CPU times: user 9.3 s, sys: 3.61 s, total: 12.9 s
Wall time: 423 ms


In [104]:
%time XtX = X_train_lift_flat.T.dot(X_train_lift_flat)

CPU times: user 9h 35min 52s, sys: 2h 40min 46s, total: 12h 16min 38s
Wall time: 24min 37s


In [125]:
%time y_train_one_hot = np.eye(100)[y_train.astype('int')]

CPU times: user 12 ms, sys: 76 ms, total: 88 ms
Wall time: 84.3 ms


In [126]:
%time Xty = X_train_lift_flat.T.dot(y_train_one_hot)

CPU times: user 2min 51s, sys: 2.12 s, total: 2min 53s
Wall time: 5.93 s


In [129]:
Xty.shape

(92160, 100)

In [107]:
np.save("XtX_100_classes_conv_12_3_3_5_batches", XtX)

In [130]:
np.save("Xty_100_classes_conv_12_3_3_5_batches", Xty)

In [1]:
from opt import ls

ModuleNotFoundError: No module named 'opt'

In [143]:
reload(ls)

<module 'opt.ls' from '../opt/ls.py'>

In [ ]:
models = []
for reg in [5e-2, 0.1]:
    %time model = ls.block_kernel_solve(XtX, Xty, lambdav=reg, eval_fn=eval_fn, block_size=5000, epochs=4)
    models.append(model)

solving block 0
Epoch: 0, Block: 0, EvalFnOutput ('Train', 0.37710885273774103, 'Test', 0.29299999999999998)
solving block 1
Epoch: 0, Block: 1, EvalFnOutput ('Train', 0.41950616329842727, 'Test', 0.31900000000000001)
solving block 2
Epoch: 0, Block: 2, EvalFnOutput ('Train', 0.44762935198423431, 'Test', 0.33939999999999998)
solving block 3
Epoch: 0, Block: 3, EvalFnOutput ('Train', 0.46890529000347775, 'Test', 0.34799999999999998)
solving block 4
Epoch: 0, Block: 4, EvalFnOutput ('Train', 0.48746087561343177, 'Test', 0.35460000000000003)
solving block 5
Epoch: 0, Block: 5, EvalFnOutput ('Train', 0.50250009660342365, 'Test', 0.36080000000000001)
solving block 6
Epoch: 0, Block: 6, EvalFnOutput ('Train', 0.51571544495536925, 'Test', 0.36420000000000002)
solving block 7
Epoch: 0, Block: 7, EvalFnOutput ('Train', 0.52855210788670348, 'Test', 0.37019999999999997)
solving block 8
Epoch: 0, Block: 8, EvalFnOutput ('Train', 0.54002086633950308, 'Test', 0.376)
solving block 9
Epoch: 0, Block: 

In [146]:
len(models)

1

In [147]:
%time eval_fn(models[0])

CPU times: user 2min 4s, sys: 50.2 s, total: 2min 54s
Wall time: 7.76 s


('Train', 0.62425905174079366, 'Test', 0.39800000000000002)

In [120]:
def eval_fn(model):
     y_pred_train = np.argmax(X_train_lift_flat.dot(model), axis=1)
     train_acc = metrics.accuracy_score(y_train, y_pred_train)
    
     y_pred_test = np.argmax(X_test_lift_flat.dot(model), axis=1)
     test_acc = metrics.accuracy_score(y_test, y_pred_test)
     return ("Train", train_acc, "Test", test_acc)

In [ ]:
for reg in [5e-2, 0.1]:
    %time acc = ls.block_kernel_solve(XTrain=X_train_lift_flat, XTest=X_test_lift_flat, labelsTrain=y_train.astype('int'), labelsTest=y_test.astype('int'), reg=reg, XTX=XtX)
    print(reg, acc)

In [1028]:
results_conv = {}
for gamma in [5e-2,1e-1, 1, 10, 100]:
    K_train_rbf_conv = linalg.computeRBFGramMatrix(X_train_lift, X_train_lift, gamma=gamma)
    K_test_rbf_conv = linalg.computeRBFGramMatrix(X_test_lift, X_train_lift, gamma=gamma)
    for reg in [1e-5, 1e-4, 1e-3, 1e-2, 1e-1]:
        acc = opt.trainAndEvaluateDualModel(KTrain=K_train_rbf_conv, KTest=K_test_rbf_conv, labelsTrain=y_train.astype('int'), 
                                                                      labelsTest=y_test.astype('int'),
                                                                      reg=reg)
        print(gamma, reg, acc)
        results[(gamma,reg)] = acc

Learning Dual Model
reg is 1e-05
0.05 1e-05 (0.99769230769230766, 0.69799999999999995)
Learning Dual Model
reg is 0.0001
0.05 0.0001 (0.98038461538461541, 0.70199999999999996)
Learning Dual Model
reg is 0.001
0.05 0.001 (0.85853846153846158, 0.70599999999999996)
Learning Dual Model
reg is 0.01
0.05 0.01 (0.72246153846153849, 0.68799999999999994)
Learning Dual Model
reg is 0.1
0.05 0.1 (0.6154615384615385, 0.59799999999999998)
Learning Dual Model
reg is 1e-05
0.1 1e-05 (0.99792307692307691, 0.68200000000000005)
Learning Dual Model
reg is 0.0001
0.1 0.0001 (0.99407692307692308, 0.70599999999999996)
Learning Dual Model
reg is 0.001
0.1 0.001 (0.92723076923076919, 0.70199999999999996)
Learning Dual Model
reg is 0.01
0.1 0.01 (0.77246153846153842, 0.68999999999999995)
Learning Dual Model
reg is 0.1
0.1 0.1 (0.65407692307692311, 0.61799999999999999)
Learning Dual Model
reg is 1e-05
1 1e-05 (0.998, 0.60599999999999998)
Learning Dual Model
reg is 0.0001
1 0.0001 (0.99807692307692308, 0.6139999

KeyboardInterrupt: 

In [ ]:
X_test_lift

In [34]:
K_test_rbf_pixels

array([[ 1.        ,  0.99926612,  0.99909939, ...,  0.99929579,
         0.99898554,  0.99931603],
       [ 0.99926612,  1.        ,  0.99913969, ...,  0.99927931,
         0.9989939 ,  0.99928697],
       [ 0.99909939,  0.99913969,  1.        , ...,  0.99925441,
         0.99911216,  0.99934219],
       ..., 
       [ 0.99905437,  0.99906212,  0.99914661, ...,  0.9995055 ,
         0.99951988,  0.99959997],
       [ 0.99926577,  0.99925152,  0.99916929, ...,  0.99948263,
         0.99917748,  0.9994573 ],
       [ 0.99912478,  0.99928218,  0.99900119, ...,  0.99943302,
         0.99906306,  0.99931051]])

In [30]:
X_train_flat.shape

(13000, 196608)

In [4]:
from utils import exputil
exputil.snapshot()

/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-12-04:01:49.198532.tar pictureweb --exclude-from <(find pictureweb -size +3M)'


In [155]:
import os
os.system("/bin/bash -c 'cd /home/ubuntu ; tar -cf ./picture_web_2017-04-12-03:53:10.611164.tar pictureweb --exclude-from <(find pictureweb -size +3M)'")

-1

In [892]:
im = imread("/home/ubuntu/tiny-imagenet-200/train/n01443537/images/n01443537_0.JPEG")

In [895]:
im.shape

(64, 64, 3)

In [923]:
fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_features_alexnet_fc7.npz")

In [924]:
X_train_fc7 = fc7["X_train"]
y_train_fc7 = fc7["y_train"]

In [925]:
X_test_fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_test_features_alexnet_fc7.npy")
y_test_fc7 = np.load("/home/ubuntu/imagenet-fc7/imagenet_test_labels.npy")

In [967]:
idxs = np.where(y_train_fc7 < 10)[0]
idxs_test = np.where(y_test_fc7 < 10)[0]
X_train_fc7_small = X_train_fc7[idxs, :]
y_train_fc7_small = y_train_fc7[idxs].astype('int')
X_test_fc7_small = X_test_fc7[idxs_test, :]
y_test_fc7_small = y_test_fc7[idxs_test].astype('int')


In [961]:
opt.trainAndEvaluatePrimalModel(X_train_fc7_small, X_test_fc7_small, y_train_fc7_small, y_test_fc7_small, reg=1000000)

X SHAPE  (12622, 4096)
Computing XTX
Done Computing XTX


(0.95642528917762637, 0.91400000000000003)

In [952]:
labels = y_train_fc7_small

In [953]:
y = np.eye(max(labels) + 1)[labels]

In [965]:
XtX = X_train_fc7_small.T.dot(X_train_fc7_small)

In [979]:
misc.rreload(softmax)
misc.rreload(opt)
from opt import softmax
softmax.softmax_gn(X_train_fc7_small, y_train_fc7_small, X_test_fc7_small, y_test_fc7_small, XtX, step_size=1)

step 1 took 0.15542125701904297
Compting train_preds 2
Iter: 0, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.9
step 1 took 0.1811075210571289
Compting train_preds 2
Iter: 1, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.9
step 1 took 0.17215442657470703
Compting train_preds 2
Iter: 2, Train Accuracy: 0.9988115987957534, Test Accuracy: 0.902
step 1 took 0.16565442085266113
Compting train_preds 2
Iter: 3, Train Accuracy: 0.9988908255427033, Test Accuracy: 0.9
step 1 took 0.1684567928314209
Compting train_preds 2
Iter: 4, Train Accuracy: 0.998970052289653, Test Accuracy: 0.9
step 1 took 0.17835235595703125
Compting train_preds 2
Iter: 5, Train Accuracy: 0.9992077325305023, Test Accuracy: 0.902
step 1 took 0.16594862937927246
Compting train_preds 2
Iter: 6, Train Accuracy: 0.9993661860244019, Test Accuracy: 0.902
step 1 took 0.16495776176452637
Compting train_preds 2
Iter: 7, Train Accuracy: 0.9994454127713516, Test Accuracy: 0.902
step 1 took 0.1683351993560791
Compting tra

KeyboardInterrupt: 

In [972]:
opt.softmax

AttributeError: module 'conv.opt' has no attribute 'softmax'